In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import re
import math

In [ ]:
tempo = pd.read_pickle('synchronization/synchronizeS')

In [ ]:
tempo

In [ ]:
identified = pd.read_pickle('identified')

In [ ]:
identified

In [ ]:
identity = identified[['seq', 'id', 'identity']]

In [ ]:
identity

In [ ]:
identity.set_index(['seq', 'id'], inplace=True)

In [ ]:
#fr = np.load('superSynchronizeQualityRGB')
bodiesDF = pd.read_pickle('synchronization/synchronize')

In [ ]:
bodiesDF.head()

In [ ]:
bodiesDF = bodiesDF.join(identity)

In [ ]:
for filename in os.listdir('/home/sabrine/notebook/framesRGB'):
    img = cv2.imread("/home/sabrine/notebook/framesRGB/" + filename)
    rgbSeq = re.search('\d+',filename).group(0)
    
    if len(bodiesDF.rgbSeq[bodiesDF.rgbSeq == int(rgbSeq)]) > 0:
        subDf = bodiesDF[bodiesDF.rgbSeq == int(rgbSeq)]
        temp = subDf.loc[subDf.index.get_level_values('seq')[0], :]
        
        for i, r in temp.iterrows():
            if r.identity != np.nan and r.identity != 'nan':
                if (isinstance(r.identity, float) and not math.isnan(r.identity)) or not isinstance(r.identity, float):
                    #detectionUpLeft = (int(bodiesDF.trackX.loc[r]), int(bodiesDF.trackY.loc[r]))
                    
                    detectionUpLeft = (int(r.trackX), int(r.trackY))
                    detectionUpLeft = (detectionUpLeft[0], detectionUpLeft[1])

                    #detectionDownRight = np.asarray((int(bodiesDF.trackX.loc[r]) + int(bodiesDF.trackW.loc[r]),
                     #                     int(bodiesDF.trackY.loc[r]) + int(bodiesDF.trackH.loc[r])))
                    detectionDownRight = (int(r.trackX + r.trackW), int(r.trackY + r.trackH))
                    detectionDownRight = (detectionDownRight[0], detectionDownRight[1])

                    if r.trackId % 3 ==0:
                        color = (255, 0, 0)
                    elif r.trackId % 3 == 1:
                        color = (0, 255, 0)
                    else:
                        color = (0, 0, 255)
                    img = cv2.rectangle(img, detectionUpLeft, detectionDownRight, color, 3)

                    centroid = tuple((np.asarray(detectionUpLeft) + np.asarray(detectionDownRight)) / 2)


                    img = cv2.circle(img, centroid, 5,  color, 2)

                    img = cv2.putText(img,str(r.identity), (int(r.trackX), int(r.trackY)), cv2.FONT_HERSHEY_PLAIN, 2, 255)
    cv2.imwrite("/home/sabrine/notebook/modified2/" + filename, img)
    

In [6]:
# To check if tracking infor are correct
for filename in os.listdir('/home/sabrine/notebook/framesRGB'):
    img = cv2.imread("/home/sabrine/notebook/framesRGB/" + filename)
    rgbSeq = re.search('\d+',filename).group(0)
    
    if len(tempo.rgbSeq[tempo.rgbSeq == int(rgbSeq)]) > 0:
        subDf = tempo[tempo.rgbSeq == int(rgbSeq)]
        temp = subDf.loc[subDf.index.get_level_values('seq')[0], :]
        
        for i, r in temp.iterrows():
            #detectionUpLeft = (int(bodiesDF.trackX.loc[r]), int(bodiesDF.trackY.loc[r]))
            detectionUpLeft = (int(r.trackX), int(r.trackY))
            detectionUpLeft = (detectionUpLeft[0], detectionUpLeft[1])
    
            #detectionDownRight = np.asarray((int(bodiesDF.trackX.loc[r]) + int(bodiesDF.trackW.loc[r]),
             #                     int(bodiesDF.trackY.loc[r]) + int(bodiesDF.trackH.loc[r])))
            detectionDownRight = (int(r.trackX + r.trackW), int(r.trackY + r.trackH))
            detectionDownRight = (detectionDownRight[0], detectionDownRight[1])
            
            if r.trackId % 3 ==0:
                color = (255, 0, 0)
            elif r.trackId % 3 == 1:
                color = (0, 255, 0)
            else:
                color = (0, 0, 255)
            img = cv2.rectangle(img, detectionUpLeft, detectionDownRight, color, 3)
                                    
            img = cv2.putText(img,str(r.trackId), (int(r.trackX), int(r.trackY)), cv2.FONT_HERSHEY_PLAIN, 2, 255)
            
    cv2.imwrite("/home/sabrine/notebook/modified/" + filename, img)